In [5]:
from gensim.models.word2vec import Word2Vec, Vocab
from gensim import utils, matutils
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from collections import defaultdict
from copy import copy
from sklearn.externals import joblib

In [6]:
infile = "wikipedia_sentences_tokenised.txt"

In [7]:
model = Word2Vec.load_word2vec_format("/Users/tejus/Downloads/glove/glove.6B.300d.txt", binary=False)

In [8]:
scws_ratings_file = "wordlist.txt"

sims = defaultdict(list)
vocab = set([])
scores_list = []
sim_list = []

for line_num, line in enumerate(open(scws_ratings_file)):
    tokens = line.split(".")
    #print tokens
    w1 = tokens[0]
    print w1
    #print w2
    #sent1, sent2 = tokens[5: 7]
    #score = float(tokens[7])

    if w1 in model.vocab:
    #    sims[w1].append((w2, score))
     #   sims[w2].append((w1, score))
        vocab.add(w1)
        # vocab.add(w2)

print vocab

art
authority
bar
blind
bum
chair
channel
child
church
circuit
colourless
cool
day
detention
dyke
facility
faithful
fatigue
feeling
fine
fit
free
graceful
green
grip
hearth
holiday
lady
local
material
mouth
nation
natural
nature
oblique
post
restraint
sense
simple
solemn
spade
stress
vital
yew
begin
call
carry
collaborate
develop
draw
dress
drift
drive
face
ferret
find
keep
leave
live
match
play
pull
replace
see
serve
strike
train
treat
turn
use
wander
wash
work
set(['blind', 'art', 'serve', 'facility', 'simple', 'oblique', 'bum', 'draw', 'authority', 'replace', 'wash', 'see', 'church', 'carry', 'graceful', 'chair', 'keep', 'fine', 'dyke', 'spade', 'use', 'develop', 'faithful', 'fit', 'solemn', 'dress', 'find', 'live', 'vital', 'call', 'yew', 'strike', 'holiday', 'local', 'channel', 'treat', 'grip', 'begin', 'collaborate', 'sense', 'nature', 'restraint', 'material', 'drift', 'detention', 'free', 'nation', 'stress', 'ferret', 'train', 'fatigue', 'colourless', 'child', 'natural', 'post',

In [9]:
len(vocab)

73

In [10]:
idf_values = dict()
for line in open("idf_values.txt"):
    word, value = line.strip().split("\t")
    idf_values[word] = float(value)

In [11]:
context_dict = defaultdict(list)
wiki_lines = []
line_id = -1

for line_num, line in enumerate(open(infile)):
    line = line.lower()
    if line_num % 1000000 == 0:
        print line_num
    
    sent_words_set = set(line.strip().split())
    new_set = sent_words_set.intersection(vocab)
    
    if new_set:
        line_id += 1
        wiki_lines.append(line)
        for word in new_set:
            context_dict[word].append(line_id)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
68000000
69000000
70000000
71000000
72000000
73000000
74000000
75000000
76000000
77000000
78000000
79000000
80000000
81000000
82000000
83000000
84000000
85000000
86000000
87000000
88000000
89000000
90000000
91000000
92000000
93000000
94000000
95000000
96000000
97000000
98000000
99000000
100000000
101000000
102000000
103000000
104000000
105000000


In [12]:
word2index = dict()
for index, word in enumerate(model.index2word):
    word2index[word] = index 

In [13]:
stop_set = set()
for line in open("stopwords.txt"):
    stop_set.add(line.strip())

In [14]:
model.syn0norm = normalize(model.syn0, axis=1)

In [15]:
#context_dict['holy']

In [16]:
def preprocessing(line):
    line = line.strip()
    line = line.replace(".", "")
    line = line.replace(",", "")
    line = line.replace('"','')
    
    #regex = r"(?u)\b\w\w+\b"

    words_list = []
    for word in line.split():
        if word not in stop_set:
            words_list.append(word)

    #return re.findall(regex, doc)
    return words_list

In [17]:
window = 2
num_clusters = 5
topn = 5
word_centroids = defaultdict(list)
target_words_completed = set()

In [23]:
for j_index, t_word in enumerate(context_dict):
    if t_word in target_words_completed:
        continue
    
    print "{}, Calculating for word = {}".format(j_index, t_word)
    context_vectors_list = list()
    cluster_centroids_list = []
    
    for line_id in context_dict[t_word]:

        line = wiki_lines[line_id]
        words_list = preprocessing(line)    

        if( len(words_list) < 5 ):
            continue

        for i, word in enumerate(words_list):
            if word == t_word:
                sum_vector = np.zeros(300)
                total_context_words = 0

                left_window = words_list[max(i - window, 0): i]
                left_index = list(range(1, len(left_window) + 1))
                left_index.reverse()
                left_window_2 = zip(left_index, left_window)

                right_window = words_list[i + 1: i + (window + 1)]
                right_index = list(range(1, len(right_window) + 1))
                right_window_2 = zip(right_index, right_window)

                aggregate_list = left_window_2 + right_window_2

                for weight, context_word in aggregate_list:
                    if context_word in word2index:
                        total_context_words += 1
                        context_word_vector = model.syn0norm[word2index[context_word]]
                        current_word_vector = model.syn0norm[word2index[word]]
                        similarity = np.dot(context_word_vector, current_word_vector)

                        if context_word in idf_values:
                            sum_vector += idf_values[context_word] * similarity * context_word_vector

                if total_context_words > 0:
                    #avg_vector = (1. / total_context_words) * (sum_vector)
                    avg_vector = sum_vector
                    context_vectors_list.append(avg_vector)

    if len(context_vectors_list) == 0:
        target_words_completed.add(t_word)
        continue
    
    context_vectors_array = normalize(np.array(context_vectors_list))
    
    if len(context_vectors_array > 5000):
        context_vectors_subset = context_vectors_array[np.random.choice(context_vectors_array.shape[0], size=500, replace=False), :]
    else:
        context_vectors_subset = context_vectors_array
        
    kmeans = KMeans(n_clusters=num_clusters, n_jobs=1, n_init=2).fit(context_vectors_subset)

    for j in range(num_clusters):
        flag = False
        dists = np.dot(model.syn0, kmeans.cluster_centers_[j])
        best = matutils.argsort(dists, topn=topn, reverse=True)

        # ignore (don't return) words from the input
        result = []
        for sim in best:
            word = model.index2word[sim]
            if word in stop_set:
                flag = True
                break
            result.append( word + " " + str(float(dists[sim])))

        if flag:
            continue
        
        cluster_centroids_list.append(kmeans.cluster_centers_[j])
        #print "\n\n", "Cluster {}".format(j)
        #print "\n".join(result[:topn])
    word_centroids[t_word] = copy(cluster_centroids_list)
    joblib.dump(np.array(cluster_centroids_list), "wikipedia_context_centroids_senseval/{}.pkl".format(t_word) )
    target_words_completed.add(t_word)
#     clf = joblib.load("wikipedia_context_centroids_senseval/{}.pkl".format(t_word))
#     print clf

49, Calculating for word = colourless
50, Calculating for word = child
51, Calculating for word = bar
52, Calculating for word = post
53, Calculating for word = lady
54, Calculating for word = day
55, Calculating for word = cool
56, Calculating for word = pull
57, Calculating for word = stress
58, Calculating for word = natural
59, Calculating for word = work
60, Calculating for word = drift
61, Calculating for word = drive
62, Calculating for word = draw
63, Calculating for word = keep
64, Calculating for word = leave
65, Calculating for word = turn
66, Calculating for word = green
67, Calculating for word = match
68, Calculating for word = circuit
69, Calculating for word = bum
70, Calculating for word = dyke
71, Calculating for word = fatigue
72, Calculating for word = feeling


In [24]:
target_words_completed

{'art',
 'authority',
 'bar',
 'begin',
 'blind',
 'bum',
 'call',
 'carry',
 'chair',
 'channel',
 'child',
 'church',
 'circuit',
 'collaborate',
 'colourless',
 'cool',
 'day',
 'detention',
 'develop',
 'draw',
 'dress',
 'drift',
 'drive',
 'dyke',
 'face',
 'facility',
 'faithful',
 'fatigue',
 'feeling',
 'ferret',
 'find',
 'fine',
 'fit',
 'free',
 'graceful',
 'green',
 'grip',
 'hearth',
 'holiday',
 'keep',
 'lady',
 'leave',
 'live',
 'local',
 'match',
 'material',
 'mouth',
 'nation',
 'natural',
 'nature',
 'oblique',
 'play',
 'post',
 'pull',
 'replace',
 'restraint',
 'see',
 'sense',
 'serve',
 'simple',
 'solemn',
 'spade',
 'stress',
 'strike',
 'train',
 'treat',
 'turn',
 'use',
 'vital',
 'wander',
 'wash',
 'work',
 'yew'}